INDEXADOR

In [ ]:
import nltk
from collections import defaultdict
from urllib.parse import urlparse
import math
import chardet

nltk.download('punkt', quiet=True)
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

class Indexador:
    def __init__(self):
        self.indice_invertido = defaultdict(lambda: defaultdict(float))
        self.documentos = defaultdict(lambda: defaultdict(float))
        self.stop_words = set(stopwords.words('portuguese'))
        self.total_documentos = 0

    def processar_texto(self, texto):
        # Tokeniza e remove stop words
        palavras = word_tokenize(texto.lower(), language='portuguese')
        return [palavra for palavra in palavras if palavra.isalpha() and palavra not in self.stop_words]

    def adicionar_documento(self, url, conteudo):
        # Detecção de codificação
        encoding = chardet.detect(conteudo.encode())['encoding']
        # Processamento do conteúdo
        palavras = self.processar_texto(conteudo)
        palavras_unicas = set(palavras)
        # Atualização do índice invertido
        for palavra in palavras_unicas:
            forca_palavra = self.avaliar_forca_palavra(palavra, conteudo)
            self.indice_invertido[palavra][url] += forca_palavra
        # Atualização dos documentos indexados
        self.documentos[url]['palavras'] = palavras
        self.documentos[url]['encoding'] = encoding
        self.total_documentos += 1

    def avaliar_forca_palavra(self, palavra, conteudo):
        # Exemplo simples: a força da palavra é o inverso do número de palavras no conteúdo
        num_palavras = len(conteudo.split())
        if num_palavras == 0:
            return 0.0
        return 1 / num_palavras

# Exemplo de uso
if __name__ == "__main__":
    indexador = Indexador()

    # Exemplo de URLs e conteúdos
    urls_conteudos = {
        "https://www.mariana.mg.gov.br/": "Conteúdo da página de Mariana",
        "https://www1.barbacena.mg.gov.br/": "Conteúdo da página de Barbacena"
    }

    # Adicionando documentos ao índice
    for url, conteudo in urls_conteudos.items():
        indexador.adicionar_documento(url, conteudo)

    # Exemplo de acesso ao índice invertido
    print("Índice Invertido:")
    for palavra, ocorrencias in indexador.indice_invertido.items():
        print(f"{palavra}: {ocorrencias}")

    # Exemplo de acesso ao número total de documentos indexados
    print(f"Total de documentos indexados: {indexador.total_documentos}")

    # Exemplo de acesso aos documentos indexados
    print("\nDocumentos indexados:")
    for url, info in indexador.documentos.items():
        print(f"URL: {url}")
        print(f"Palavras: {info['palavras']}")
        print(f"Encoding: {info['encoding']}")
        print()

Índice Invertido:
página: defaultdict(<class 'float'>, {'https://www.mariana.mg.gov.br/': 0.2, 'https://www1.barbacena.mg.gov.br/': 0.2})
conteúdo: defaultdict(<class 'float'>, {'https://www.mariana.mg.gov.br/': 0.2, 'https://www1.barbacena.mg.gov.br/': 0.2})
mariana: defaultdict(<class 'float'>, {'https://www.mariana.mg.gov.br/': 0.2})
barbacena: defaultdict(<class 'float'>, {'https://www1.barbacena.mg.gov.br/': 0.2})
Total de documentos indexados: 2

Documentos indexados:
URL: https://www.mariana.mg.gov.br/
Palavras: ['conteúdo', 'página', 'mariana']
Encoding: utf-8

URL: https://www1.barbacena.mg.gov.br/
Palavras: ['conteúdo', 'página', 'barbacena']
Encoding: utf-8

